In [1]:
from gnnwr import models,datasets
import pandas as pd
import numpy as np
import folium
import torch.nn as nn
from sklearn.metrics import r2_score as r2
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(u'../data/pm25_data.csv')
print(data.head(5))

In [ ]:
lon_center,lat_center = data['经度'].mean(),data['纬度'].mean()
map = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
data.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map),axis=1)
map

In [ ]:
train_dataset, val_dataset, test_dataset = datasets.init_dataset(data=data,
                                                        test_ratio=0.15,
                                                        valid_ratio=0.15,
                                                        x_column=['dem', 'w10','d10','t2m','aod_sat','tp'],
                                                        y_column=['PM2_5'],
                                                        spatial_column=['经度','纬度'],
                                                        sample_seed=42,
                                                        batch_size=64)

In [ ]:
map_1 = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
train_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='red'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)
val_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='green'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)
test_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='blue'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)

map_1

In [ ]:
gnnwr = models.GNNWR(train_dataset = train_dataset,
                     valid_dataset = val_dataset, 
                     test_dataset = test_dataset,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adam",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result//gnnwr_logs",
                     write_path = "./demo_result/gnnwr_tensorboard/"
                     )

In [ ]:
gnnwr.run(max_epoch = 100,early_stop = 5000,print_frequency = 1000)

In [ ]:
gnnwr.result()

In [ ]:
gnnwr.reg_result('./demo_result/GNNWR_PM25_Result.csv')

In [ ]:
result_data = pd.read_csv('./demo_result/GNNWR_PM25_Result.csv')
result_data['id'] = result_data['id'].astype(np.int64)
result_data.rename(columns={"PM2_5":"Pred_PM2_5"},inplace=True)
data = pd.concat([train_dataset.dataframe,val_dataset.dataframe,test_dataset.dataframe])
data.head(5)
data.set_index('id',inplace=True)
result_data.set_index('id',inplace=True)
result_data = result_data.join(data)
result_data.head(5)

In [ ]:
print(train_dataset.distances_scale_param['min'].tolist())

In [ ]:
train_dataset.save('./demo_result/gnnwr_datasets/train_dataset')
val_dataset.save('./demo_result/gnnwr_datasets/val_dataset')
test_dataset.save('./demo_result/gnnwr_datasets/test_dataset')

In [2]:
train_dataset_loaded = datasets.load_dataset('./demo_result/gnnwr_datasets/train_dataset/')
val_dataset_loaded = datasets.load_dataset('./demo_result/gnnwr_datasets/val_dataset/')
test_dataset_loaded = datasets.load_dataset('./demo_result/gnnwr_datasets/test_dataset/')

In [ ]:
print(val_dataset.batch_size)

In [ ]:
print(val_dataset_loaded.shuffle)

In [3]:
gnnwr_loaded = models.GNNWR(train_dataset = train_dataset_loaded,
                     valid_dataset = val_dataset_loaded, 
                     test_dataset = test_dataset_loaded,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adam",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result//gnnwr_logs",
                     write_path = "./demo_result/gnnwr_tensorboard/"
                     )

In [4]:
gnnwr_loaded.load_model('./demo_result/gnnwr_models/GNNWR_PM25.pkl')

In [5]:
gnnwr_loaded.result()

Test Loss:  55.986881256103516  Test R2:  0.7312301287858805
--------------------Result Table--------------------

Model Name:           | GNNWR_PM25
Model Structure:      |
 SWNN(
  (activate_func): PReLU(num_parameters=1)
  (fc): Sequential(
    (swnn_full0): Linear(in_features=1017, out_features=512, bias=True)
    (swnn_batc0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti0): PReLU(num_parameters=1)
    (swnn_drop0): Dropout(p=0.2, inplace=False)
    (swnn_full1): Linear(in_features=512, out_features=256, bias=True)
    (swnn_batc1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti1): PReLU(num_parameters=1)
    (swnn_drop1): Dropout(p=0.2, inplace=False)
    (swnn_full2): Linear(in_features=256, out_features=128, bias=True)
    (swnn_batc2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti2): PReLU(num_parameters=1)
    (swnn_drop2): Dropo